In [1]:
import numpy as np
import tritonclient.http as httpclient

# Prepare inputs

In [2]:
client = httpclient.InferenceServerClient('localhost:8000')

In [3]:
inputs= [
    httpclient.InferInput(
        "input_ids",
        [1, 512],
        "INT64"
    ),
    httpclient.InferInput(
        "attention_mask",
        [1, 512],
        "INT64"
    ),
    httpclient.InferInput(
        "token_type_ids",
        [1, 512],
        "INT64"
    ),
    httpclient.InferInput(
        "bbox",
        [1, 512, 4],
        "INT64"
    ),
]

outputs = [
    httpclient.InferRequestedOutput('output', binary_data=True)
]

In [4]:
inputs[0].set_data_from_numpy(np.random.randint(0, 1, (1, 512)))
inputs[1].set_data_from_numpy(np.random.randint(0, 1, (1, 512)))
inputs[2].set_data_from_numpy(np.random.randint(0, 1, (1, 512)))
inputs[3].set_data_from_numpy(np.random.randint(0, 1, (1, 512, 4)))

# Run inference

In [12]:
results = client.infer(
    'layoutlm_onnx',inputs, outputs=outputs,    
)

In [13]:
preds = results.as_numpy('output')

In [14]:
preds = np.argmax(preds, axis=1)

In [15]:
preds

array([[  0, 129]])